<a href="https://colab.research.google.com/github/Vivek-Sajjan/Conversational-AI-Project/blob/main/twitter_analysis_Delta_mongo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd

In [11]:
!wget https://raw.githubusercontent.com/Vivek-Sajjan/Conversational-AI-Project/main/Dataset/Delta_Airline_tweets.csv

--2025-09-27 12:32:19--  https://raw.githubusercontent.com/Vivek-Sajjan/Conversational-AI-Project/main/Dataset/Delta_Airline_tweets.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5771978 (5.5M) [text/plain]
Saving to: ‘Delta_Airline_tweets.csv.1’

Delta_Airline_tweet 100%[===================>]   5.50M  --.-KB/s    in 0.08s   

2025-09-27 12:32:19 (71.8 MB/s) - ‘Delta_Airline_tweets.csv.1’ saved [5771978/5771978]



In [12]:
df = pd.read_csv('Delta_Airline_tweets.csv')
display(df.head())

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,609,Delta,False,Sat Aug 06 01:44:03 +0000 2016,"@115818 Glad to check. Pls, DM your confirmati...",610,611.0
1,610,115818,True,Tue Oct 31 22:11:33 +0000 2017,@Delta DM sent,NaN,609.0
2,611,115818,True,Sat Aug 06 01:31:50 +0000 2016,@DELTA i booked my flight using delta amex car...,609,NaN
3,790,Delta,False,Tue Oct 31 22:27:38 +0000 2017,@115882 I'm sorry. The earlier flight may not ...,791,792.0
4,791,115882,True,Tue Oct 31 22:33:11 +0000 2017,@Delta I checked daily and flight 1403 was nev...,793,790.0


In [13]:
df.shape

(34315, 7)

In [14]:
# Create a new 'role' column based on the 'inbound' column
df['role'] = df['inbound'].apply(lambda x: 'Customer' if x else 'Service Provider')

# Get the list of existing columns
cols = df.columns.tolist()

# Move the 'role' column to the second position (index 2)
cols.insert(2, cols.pop(cols.index('role')))

# Reindex the DataFrame with the new column order
df = df[cols]

# Display the head of the modified DataFrame
display(df.head())

,tweet_id,author_id,role,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,609,Delta,Service Provider,False,Sat Aug 06 01:44:03 +0000 2016,"@115818 Glad to check. Pls, DM your confirmati...",610,611.0
1,610,115818,Customer,True,Tue Oct 31 22:11:33 +0000 2017,@Delta DM sent,NaN,609.0
2,611,115818,Customer,True,Sat Aug 06 01:31:50 +0000 2016,@DELTA i booked my flight using delta amex car...,609,NaN
3,790,Delta,Service Provider,False,Tue Oct 31 22:27:38 +0000 2017,@115882 I'm sorry. The earlier flight may not ...,791,792.0
4,791,115882,Customer,True,Tue Oct 31 22:33:11 +0000 2017,@Delta I checked daily and flight 1403 was nev...,793,790.0


In [19]:
# Get the list of existing columns
cols = df.columns.tolist()

# Remove the original 'created_date', and 'created_time' columns from the list
cols.remove('created_date')
cols.remove('created_time')

# Find the index where you want to insert the new columns (e.g., after 'role')
# Assuming 'role' is at index 2 after the previous modifications
insert_pos = cols.index('inbound')

# Insert the new columns into the desired position
cols.insert(insert_pos, 'created_date')
cols.insert(insert_pos + 1, 'created_time')

# Reindex the DataFrame with the new column order
df = df[cols]

# Display the head of the modified DataFrame
display(df.head())

,tweet_id,author_id,role,created_date,created_time,inbound,text,response_tweet_id,in_response_to_tweet_id
0,609,Delta,Service Provider,2016-08-06,01:44:03,False,"@115818 Glad to check. Pls, DM your confirmati...",610,611.0
1,610,115818,Customer,2017-10-31,22:11:33,True,@Delta DM sent,NaN,609.0
2,611,115818,Customer,2016-08-06,01:31:50,True,@DELTA i booked my flight using delta amex car...,609,NaN
3,790,Delta,Service Provider,2017-10-31,22:27:38,False,@115882 I'm sorry. The earlier flight may not ...,791,792.0
4,791,115882,Customer,2017-10-31,22:33:11,True,@Delta I checked daily and flight 1403 was nev...,793,790.0


In [24]:
from datetime import datetime

# Ensure 'created_date' is in datetime format
df['created_date'] = pd.to_datetime(df['created_date'], errors='coerce')

# Change the year to 2024
# Using errors='coerce' in to_datetime handles any potential parsing issues by turning them into NaT
df['created_date'] = df['created_date'].apply(lambda x: x.replace(year=2024) if pd.notnull(x) else None)

# Display the head of the DataFrame to show the updated 'created_date' column
display(df.head())

,tweet_id,author_id,role,created_date,created_time,inbound,text,response_tweet_id,in_response_to_tweet_id
0,609,Delta,Service Provider,2024-08-06,01:44:03,False,"@115818 Glad to check. Pls, DM your confirmati...",610,611.0
1,610,115818,Customer,2024-10-31,22:11:33,True,@Delta DM sent,NaN,609.0
2,611,115818,Customer,2024-08-06,01:31:50,True,@DELTA i booked my flight using delta amex car...,609,NaN
3,790,Delta,Service Provider,2024-10-31,22:27:38,False,@115882 I'm sorry. The earlier flight may not ...,791,792.0
4,791,115882,Customer,2024-10-31,22:33:11,True,@Delta I checked daily and flight 1403 was nev...,793,790.0


In [28]:
# Rename the column
df = df.rename(columns={'created_date': 'created_at'})

# Display the head of the DataFrame to show the renamed column
display(df.head())

,tweet_id,author_id,role,created_at,created_time,inbound,text,response_tweet_id,in_response_to_tweet_id
0,609,Delta,Service Provider,2024-08-06,01:44:03,False,"@115818 Glad to check. Pls, DM your confirmati...",610,611.0
1,610,115818,Customer,2024-10-31,22:11:33,True,@Delta DM sent,NaN,609.0
2,611,115818,Customer,2024-08-06,01:31:50,True,@DELTA i booked my flight using delta amex car...,609,NaN
3,790,Delta,Service Provider,2024-10-31,22:27:38,False,@115882 I'm sorry. The earlier flight may not ...,791,792.0
4,791,115882,Customer,2024-10-31,22:33:11,True,@Delta I checked daily and flight 1403 was nev...,793,790.0


# Task
Analyze the "Delta_Airline" file and build the conversations.

## Identify conversation starters

### Subtask:
Find the initial tweets that start a conversation (those with no `in_response_to_tweet_id`).


**Reasoning**:
Filter the dataframe to find tweets with no response, which signify the start of a conversation. Store the results in a new dataframe and display the head.



In [29]:
conversation_starters_df = df[df['in_response_to_tweet_id'].isnull()]
display(conversation_starters_df.head())

,tweet_id,author_id,role,created_at,created_time,inbound,text,response_tweet_id,in_response_to_tweet_id
2,611,115818,Customer,2024-08-06,01:31:50,True,@DELTA i booked my flight using delta amex car...,609,NaN
6,792,115882,Customer,2024-10-31,21:33:48,True,@Delta why wasn't earlier flight offered when ...,790,NaN
13,801,115883,Customer,2024-10-29,15:59:35,True,"@Delta The ""change flight"" search option on yo...",799,NaN
15,803,115884,Customer,2024-10-31,21:33:27,True,.@delta this has been my inflight studio exper...,802,NaN
24,814,115885,Customer,2024-10-31,16:37:26,True,@Delta I'm flying JFK-MEX-MID tomorrow and you...,813,NaN


In [30]:
conversation_starters_df.shape

(10150, 9)

## Trace conversations

### Subtask:
For the first few conversation starters, trace the subsequent tweets using `tweet_id`, `response_tweet_id`, and `in_response_to_tweet_id` to build complete conversation threads.


**Reasoning**:
Iterate through the all the conversation starters and trace their replies to build complete conversation threads.



In [31]:
conversations = []
for index, starter_tweet in conversation_starters_df.iterrows():
    conversation = [starter_tweet]
    current_tweet_id = starter_tweet['tweet_id']

    while True:
        next_tweet = df[df['in_response_to_tweet_id'] == current_tweet_id]
        if not next_tweet.empty:
            # Assuming there's only one reply per tweet in a simple thread
            current_tweet = next_tweet.iloc[0]
            conversation.append(current_tweet)
            current_tweet_id = current_tweet['tweet_id']
        else:
            break
    conversations.append(conversation)

# Display the first conversation as an example
if conversations:
    print("First Conversation Thread:")
    for tweet in conversations[0]:
        display(tweet)

First Conversation Thread:


,2
tweet_id,611
author_id,115818
role,Customer
created_at,2024-08-06 00:00:00
created_time,01:31:50
inbound,True
text,@DELTA i booked my flight using delta amex car...
response_tweet_id,609
in_response_to_tweet_id,NaN


,0
tweet_id,609
author_id,Delta
role,Service Provider
created_at,2024-08-06 00:00:00
created_time,01:44:03
inbound,False
text,"@115818 Glad to check. Pls, DM your confirmati..."
response_tweet_id,610
in_response_to_tweet_id,611.0


,1
tweet_id,610
author_id,115818
role,Customer
created_at,2024-10-31 00:00:00
created_time,22:11:33
inbound,True
text,@Delta DM sent
response_tweet_id,NaN
in_response_to_tweet_id,609.0


## Structure the conversations

### Subtask:
Organize the identified tweets into the distinct conversation structures.


**Reasoning**:
Create a list to store the structured conversations and iterate through the traced conversations to organize them into a structured format.



In [35]:
import json
import numpy as np

# Custom encoder to handle NumPy types, including booleans
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.bool_):
            return bool(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        # Add handling for pandas Series to convert them to dictionary
        if isinstance(obj, pd.Series):
            return obj.to_dict()
        return super(NpEncoder, self).default(obj)

structured_conversations = []

for i, conversation_thread in enumerate(conversations):  # Process all conversations
    structured_thread = {
        'conversation_number': i + 1,
        'customer': 'Delta',
        'tweets': []
    }
    for tweet in conversation_thread:
        structured_thread['tweets'].append({
            'tweet_id': tweet['tweet_id'],
            'author_id': tweet['author_id'],
            'role': tweet['role'],
            'inbound': tweet['inbound'],
            'created_at': tweet['created_at'],
            'text': tweet['text']
        })
    structured_conversations.append(structured_thread)

# Display the first structured conversation as an example
if structured_conversations:
    print("First Structured Conversation:")
    # Print the dictionary representation for clarity
    print(json.dumps(structured_conversations[0], indent=4, cls=NpEncoder))

First Structured Conversation:
{
    "conversation_number": 1,
    "customer": "Delta",
    "tweets": [
        {
            "tweet_id": 611,
            "author_id": "115818",
            "role": "Customer",
            "inbound": true,
            "created_at": "2024-08-06T00:00:00",
            "created_time": "01:31:50",
            "text": "@DELTA i booked my flight using delta amex card. Checking in now &amp; was being charged for baggage"
        },
        {
            "tweet_id": 609,
            "author_id": "Delta",
            "role": "Service Provider",
            "inbound": false,
            "created_at": "2024-08-06T00:00:00",
            "created_time": "01:44:03",
            "text": "@115818 Glad to check. Pls, DM your confirmation number for assistance.  *QB https://t.co/6iDGBJAc2m"
        },
        {
            "tweet_id": 610,
            "author_id": "115818",
            "role": "Customer",
            "inbound": true,
            "created_at": "2024-10-31

In [36]:
!pip install pytz

In [41]:
import random
import json
import numpy as np
from datetime import datetime
import pytz

# Get current time in Indian Standard Time (IST)
ist = pytz.timezone('Asia/Kolkata')
current_time_ist = datetime.now(ist)

# Generate filename with current date and time (IST) in requested format
formatted_time = current_time_ist.strftime('%Y%m%d_%H%M%S')
output_filename = f'Delta_Airline_100_Dec2024_{formatted_time}.json'


# Custom encoder to handle NumPy types, including booleans and datetime
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.bool_):
            return bool(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        # Add handling for pandas Series to convert them to dictionary
        if isinstance(obj, pd.Series):
            return obj.to_dict()
        # Add handling for datetime objects (including pandas Timestamp which is a subclass)
        if isinstance(obj, datetime):
            # Format as YYYY-MM-DD string
            return obj.strftime('%Y-%m-%d')
        # Handle time objects
        if isinstance(obj, datetime.time):
            return obj.isoformat()
        return super(NpEncoder, self).default(obj)


# Filter conversations for December 2024
dec_2024_conversations = []
for conversation in structured_conversations:
    # Check if any tweet in the conversation is from Dec 2024
    if any(isinstance(tweet['created_at'], datetime) and
           tweet['created_at'].year == 2024 and
           tweet['created_at'].month == 12
           for tweet in conversation['tweets']):
        dec_2024_conversations.append(conversation)


# Select 100 random conversations from the filtered list
if len(dec_2024_conversations) >= 100:
    random_dec_2024_conversations = random.sample(dec_2024_conversations, 100)
else:
    random_dec_2024_conversations = dec_2024_conversations
    print(f"Warning: Only {len(dec_2024_conversations)} conversations found for December 2024. Saving all of them.")


# Save the selected conversations to a JSON file
with open(output_filename, 'w') as f:
    json.dump(random_dec_2024_conversations, f, indent=4, cls=NpEncoder)

print(f"Selected {len(random_dec_2024_conversations)} random conversations from December 2024 saved to '{output_filename}'")

Selected 100 random conversations from December 2024 saved to 'Delta_Airline_100_Dec2024_20250927_183635.json'
